In [1]:
import json
import random
import math
import copy
import numpy as np
from numpy.linalg import norm
import time

import re
import razdel
import pymorphy2
from collections import Counter, namedtuple
import networkx as nx

from tqdm.notebook import tqdm
from itertools import combinations
from navec import Navec
import youtokentome as yttm
from summa.summarizer import summarize
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

import matplotlib.pyplot as plt

In [27]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [28]:
!python --version

Python 3.11.5


In [29]:
!pip3 install razdel networkx pymorphy2 nltk rouge==0.3.1 summa
!pip3 install youtokentome
!pip3 install navec
!pip3 install transformers

In [4]:
torch.cuda.is_available()

True

## EDA

In [31]:
# !wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [32]:
# !head -n 1 gazeta_train.txt
# !cat gazeta_train.txt | wc -l
# !cat gazeta_val.txt | wc -l
# !cat gazeta_test.txt | wc -l

In [33]:
!pip install pandas
!pip install numpy

In [34]:
!pip install pyarrow
!pip install fastparquet

In [5]:
import os 
import pandas as pd
import numpy as np
rootdir = "C:/Users/compf/Documents/Diploma/data/2023/master/"
os.chdir(rootdir) 

df = pd.read_parquet( 'df_ru_2.parquet.gzip' )

In [6]:
df

,AbstractHeader,AbstractSection,Arxiv_ID,Title,Abstract,paperSection,Summary
0,Полученные результаты,Мы разрабатываем мультипоследовательную процед...,http://arxiv.org/abs/cond-mat/9809415v2,Дизайн последовательностей с хорошими свойства...,Предыстория: Разработка аминокислотных последо...,Результаты решетчатой ​​модели. В этом разделе...,Полученные результаты. Мы разрабатываем мульти...
1,Фон,Прогнозирование трехмерных структур белков по ...,http://arxiv.org/abs/q-bio/0609037v3,Сравнение встречаемости и состава аминокислот ...,Предыстория: Прогнозирование трехмерных структ...,Фон. Расшифровка нативной конформации белка по...,Фон. Прогнозирование трехмерных структур белко...
2,Полученные результаты,"В данной работе мы предлагаем метод, основанны...",http://arxiv.org/abs/q-bio/0609037v3,Сравнение встречаемости и состава аминокислот ...,Предыстория: Прогнозирование трехмерных структ...,Результаты и обсуждение. Роль повторного взвеш...,Полученные результаты. В данной работе мы пред...
3,Выводы,"Линейный дискриминантный анализ, основанный на...",http://arxiv.org/abs/q-bio/0609037v3,Сравнение встречаемости и состава аминокислот ...,Предыстория: Прогнозирование трехмерных структ...,Выводы. В этой статье мы предложили простой ме...,"Выводы. Линейный дискриминантный анализ, основ..."
4,МЕТОД,СУБЪЕКТ: Биологической моделью была центральна...,http://arxiv.org/abs/0912.5238v1,Обнаружение нейронных событий ex-vivo с исполь...,Электромагнитные частоты до нескольких терагер...,Экспериментальный. . Модель животного. Наша би...,МЕТОД. СУБЪЕКТ: Биологической моделью была цен...
...,...,...,...,...,...,...,...
5480,Заключение,Мы предоставляем научно обоснованные рекоменда...,http://arxiv.org/abs/1710.04692v3,Как обратиться за технической помощью? Доказат...,Контекст: успех Stack Overflow и других общест...,. Выводы. В этой статье мы представили эмпирич...,Заключение. Мы предоставляем научно обоснованн...
5481,Выводы,STPA-Priv помогает выявить сложные риски конфи...,http://arxiv.org/abs/1710.11571v1,Предварительное исследование расширения конфид...,Контекст: Теоретико-системный анализ процессов...,VI. ОБСУЖДЕНИЕ И ВЫВОД. Это было очень краткое...,Выводы. STPA-Priv помогает выявить сложные рис...
5482,Методология,Два последовательных опроса (N= и N=) были про...,http://arxiv.org/abs/1710.11559v1,Неявные теории и самоэффективность во вводном ...,Вклад: В этом исследовании изучались усилия и ...,II. МЕТОД. В настоящем исследовании используют...,Методология. Два последовательных опроса (N= и...
5483,СЛУЧАЙ,П). Цель,http://arxiv.org/abs/1710.11564v1,Платформа киберавтомобильного моделирования и ...,Легко перемещающийся и безопасный транспорт – ...,"Введение. Желание идти туда, куда мы хотим, и ...",СЛУЧАЙ. П). Цель


In [7]:
def count_length(lst):
    lenn = 0
    for l in lst:
        lenn+=len(l)
    return lenn

In [8]:
df['len']=df['Summary'].apply(count_length)

In [9]:
df = df[df['len'] > 0]

In [10]:
df

,AbstractHeader,AbstractSection,Arxiv_ID,Title,Abstract,paperSection,Summary,len
0,Полученные результаты,Мы разрабатываем мультипоследовательную процед...,http://arxiv.org/abs/cond-mat/9809415v2,Дизайн последовательностей с хорошими свойства...,Предыстория: Разработка аминокислотных последо...,Результаты решетчатой ​​модели. В этом разделе...,Полученные результаты. Мы разрабатываем мульти...,523
1,Фон,Прогнозирование трехмерных структур белков по ...,http://arxiv.org/abs/q-bio/0609037v3,Сравнение встречаемости и состава аминокислот ...,Предыстория: Прогнозирование трехмерных структ...,Фон. Расшифровка нативной конформации белка по...,Фон. Прогнозирование трехмерных структур белко...,240
2,Полученные результаты,"В данной работе мы предлагаем метод, основанны...",http://arxiv.org/abs/q-bio/0609037v3,Сравнение встречаемости и состава аминокислот ...,Предыстория: Прогнозирование трехмерных структ...,Результаты и обсуждение. Роль повторного взвеш...,Полученные результаты. В данной работе мы пред...,550
3,Выводы,"Линейный дискриминантный анализ, основанный на...",http://arxiv.org/abs/q-bio/0609037v3,Сравнение встречаемости и состава аминокислот ...,Предыстория: Прогнозирование трехмерных структ...,Выводы. В этой статье мы предложили простой ме...,"Выводы. Линейный дискриминантный анализ, основ...",472
4,МЕТОД,СУБЪЕКТ: Биологической моделью была центральна...,http://arxiv.org/abs/0912.5238v1,Обнаружение нейронных событий ex-vivo с исполь...,Электромагнитные частоты до нескольких терагер...,Экспериментальный. . Модель животного. Наша би...,МЕТОД. СУБЪЕКТ: Биологической моделью была цен...,488
...,...,...,...,...,...,...,...,...
5480,Заключение,Мы предоставляем научно обоснованные рекоменда...,http://arxiv.org/abs/1710.04692v3,Как обратиться за технической помощью? Доказат...,Контекст: успех Stack Overflow и других общест...,. Выводы. В этой статье мы представили эмпирич...,Заключение. Мы предоставляем научно обоснованн...,355
5481,Выводы,STPA-Priv помогает выявить сложные риски конфи...,http://arxiv.org/abs/1710.11571v1,Предварительное исследование расширения конфид...,Контекст: Теоретико-системный анализ процессов...,VI. ОБСУЖДЕНИЕ И ВЫВОД. Это было очень краткое...,Выводы. STPA-Priv помогает выявить сложные рис...,254
5482,Методология,Два последовательных опроса (N= и N=) были про...,http://arxiv.org/abs/1710.11559v1,Неявные теории и самоэффективность во вводном ...,Вклад: В этом исследовании изучались усилия и ...,II. МЕТОД. В настоящем исследовании используют...,Методология. Два последовательных опроса (N= и...,454
5483,СЛУЧАЙ,П). Цель,http://arxiv.org/abs/1710.11564v1,Платформа киберавтомобильного моделирования и ...,Легко перемещающийся и безопасный транспорт – ...,"Введение. Желание идти туда, куда мы хотим, и ...",СЛУЧАЙ. П). Цель,16


In [11]:
!pip install scikit-learn

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['paperSection'], df['Summary'], test_size=0.3, random_state=42)

In [43]:
# def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
#     assert shuffle != sort_by_date
#     records = []
#     with open(file_name, "r") as r:
#         for line in r:
#             records.append(json.loads(line))
#     if sort_by_date:
#         records.sort(key=lambda x: x["date"])
#     if shuffle:
#         random.shuffle
#     return records

In [44]:
# train_records = read_gazeta_records("gazeta_train.txt")
# val_records = read_gazeta_records("gazeta_val.txt")
# test_records = read_gazeta_records("gazeta_test.txt")

In [45]:
# train_records[0].keys()

In [46]:
# print(min([record["date"] for record in train_records]), end=' - ')
# print(max([record["date"] for record in train_records]))
# print(min([record["date"] for record in val_records]), end=' - ')
# print(max([record["date"] for record in val_records]))
# print(min([record["date"] for record in test_records]), end=' - ')
# print(max([record["date"] for record in test_records]))

In [13]:
Stats = namedtuple("Stats", "vocabulary,lemma_vocabulary,words_counts,unique_words_counts")

def collect_stats(records, lower=True, text_max_words=3000, summary_max_words=100, nrows=3810):
    morph = pymorphy3.MorphAnalyzer()

    text_stats = Stats(Counter(),  Counter(), list(), list())
    summary_stats = Stats(Counter(),  Counter(), list(), list())

    def update_record_field_stats(field, stats, max_words):
        words = [word.text for word in razdel.tokenize(field)][:max_words]
        lemmas = [morph.parse(word)[0].normal_form for word in words]
        stats.vocabulary.update(words)
        stats.lemma_vocabulary.update(lemmas)
        stats.words_counts.append(len(words))
        stats.unique_words_counts.append(len(set(words)))

    for i, record in tqdm(records.iterrows()):
        
        if i >= nrows:
            break
        
        text = record["text"]
        text = text if not lower else text.lower()
        update_record_field_stats(text, text_stats, text_max_words)

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        summary_words = [word.text for word in razdel.tokenize(summary)]
        update_record_field_stats(summary, summary_stats, summary_max_words)
    return text_stats, summary_stats

In [14]:
frames = [X_train,y_train]

In [15]:
train_records = pd.concat(frames, axis=1)
train_records.columns =['text', 'summary']
train_records = train_records.reset_index(drop=True)
# train_records = train_records.drop('index', axis=1)

In [16]:
train_records

,text,summary
0,"VII. ВЫВОДЫ. Наши результаты ясно показывают, ...",Выводы. Изоскалярный бозон не улучшает энергет...
1,. Полученные результаты. . .. Новые данные. Мы...,Полученные результаты. Мы обнаруживаем источни...
2,. Резюме и выводы. На основе глобальных астеро...,"Выводы. И масса, и металличность способствуют ..."
3,. Выводы. Наш верхний предел содержания серы н...,Выводы. Этот верхний предел не дает убедительн...
4,. Полученные результаты. . .. Холодный и теплы...,"Полученные результаты. Одиночные, близкие двой..."
...,...,...
3834,. Выводы. В этой статье представлены значения ...,Выводы. Измеренные MBSF и полученные коэффицие...
3835,МАТЕРИАЛЫ И МЕТОДЫ. . Корпоративные и кандидат...,Методы. Мы разработали инновационную адаптиров...
3836,". ОБСУЖДЕНИЕ. Оценки, представленные в предыду...",Обсуждение. Оба подхода дали весьма обнадежива...
3837,II. МАТЕРИАЛЫ И МЕТОДЫ. . А. Сбор данных. Набо...,Методы. Объемы -D ОКМ высокого разрешения (каж...


In [17]:
frames = [X_test,y_test]

In [18]:
test_records = pd.concat(frames, axis=1)
test_records.columns =['text', 'summary']
test_records = test_records.reset_index(drop=True)
# train_records = train_records.drop('index', axis=1)

In [19]:
test_records

,text,summary
0,. Полученные результаты. . .. Суммарная числен...,Полученные результаты. Лучший подход для SP — ...
1,ОБСУЖДЕНИЕ И ВЫВОДЫ. Измерение образования мюо...,Выводы. В пределах экспериментальных и теорети...
2,. Выводы. Эта работа подтверждает наблюдение S...,Выводы. Мы представили функциональное поведени...
3,Полученные результаты. . . Этапы МОЭ. Популяци...,"Полученные результаты. Методы ускорения, реали..."
4,. Полученные результаты. . .. Группа . Мы иска...,Полученные результаты. Мы проводим опрос канди...
...,...,...
1641,РЕЗУЛЬТАТЫ: ХАРАКТЕРИСТИКИ НАСЕЛЕНИЯ ВОЙДЫ. В ...,Полученные результаты. Мы подтверждаем статист...
1642,. Заключение. Зависимость амплитуд от периода ...,"Выводы. Взаимозависимости амплитуд пульсаций, ..."
1643,. Методы. . . Теоретические модели. Количестве...,Методы. Улучшение производительности современн...
1644,. Выводы. В ходе миссии «Кампания К» мы обнару...,Выводы. Мы нашли планету в . AU от одного тран...


In [54]:
!pip install pymorphy3

In [20]:
import pymorphy3

In [21]:
train_text_stats, train_summary_stats = collect_stats(train_records)
print("Train texts vocabulary size: ", len(train_text_stats.vocabulary))
print("Train texts lemma vocabulary size: ", len(train_text_stats.lemma_vocabulary))
print("Train summaries vocabulary size: ", len(train_summary_stats.vocabulary))
print("Train summaries lemma vocabulary size: ", len(train_summary_stats.lemma_vocabulary))
print("Train common lemmas summary vs text: ", len(set(train_text_stats.lemma_vocabulary.keys()) & set(train_summary_stats.lemma_vocabulary.keys())))

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 5))
axs[0].hist(train_text_stats.words_counts, 20)
axs[0].set_title('# Words in texts')

axs[1].hist(train_summary_stats.words_counts, 20)
axs[1].set_title('# Words in summary')
plt.show()

## Metrics

In [27]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

#TextRank

In [59]:
def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    '''
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2)) / (len(words1) + len(words2))
    #return len(words1.intersection(words2)) / (np.log10(len(words1)) + np.log10(len(words2)))

def gen_text_rank_summary(text, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank

    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    num_sents = len(result)
    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary, num_sents

def calc_text_rank_score(records, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, nrows=1620, morph=None):
    references = []
    predictions = []
    length_sents = []

    for i, record in tqdm(records.iterrows()):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        predicted_summary, num_sents = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=morph)
        length_sents.append(num_sents)
        text = text if not lower else text.lower()
        
        
        if len(predicted_summary)==0:
            predicted_summary='None'
        
        predictions.append(predicted_summary)
    
    print(f'Mean number of sents in summary: {np.mean(length_sents)}')
#     print(references[143])
#     print(predictions[143])
    calc_scores(references, predictions)

morph = pymorphy3.MorphAnalyzer()
calc_text_rank_score(test_records)

0it [00:00, ?it/s]

Mean number of sents in summary: 1.9012345679012346
Count: 1620
Ref: методы. мы оценили два современных метода, все из которых основаны на использовании глубоких сверточных нейронных сетей и их способности прогнозировать количество митозов на слайдах цифровой гистопатологии. мы оценили их на новом наборе данных, состоящем из полностью аннотированных цельных изображений тучных клеток кожи собак (cmct) и одного общедоступного набора данных о карциноме молочной железы человека (hmc). сначала мы сравнили количество митозов (mc), предсказанное двумя моделями, с фактическим mc в обоих наборах данных. во-вторых, для набора данных cmct мы сравнили предсказанное с помощью вычислений положение и mc области наивысшей митотической активности с эквивалентными по размеру областями, выбранными восемью ветеринарными патологами.
Hyp: соотношение сторон области такое же, как и для алгоритмического конвейера (:).. . общий алгоритмический подход.
BLEU:  0.09501924420322916
ROUGE:  {'rouge-1': {'f': 0.1044

In [60]:
test_records

,text,summary
0,. Полученные результаты. . .. Суммарная числен...,Полученные результаты. Лучший подход для SP — ...
1,ОБСУЖДЕНИЕ И ВЫВОДЫ. Измерение образования мюо...,Выводы. В пределах экспериментальных и теорети...
2,. Выводы. Эта работа подтверждает наблюдение S...,Выводы. Мы представили функциональное поведени...
3,Полученные результаты. . . Этапы МОЭ. Популяци...,"Полученные результаты. Методы ускорения, реали..."
4,. Полученные результаты. . .. Группа . Мы иска...,Полученные результаты. Мы проводим опрос канди...
...,...,...
1641,РЕЗУЛЬТАТЫ: ХАРАКТЕРИСТИКИ НАСЕЛЕНИЯ ВОЙДЫ. В ...,Полученные результаты. Мы подтверждаем статист...
1642,. Заключение. Зависимость амплитуд от периода ...,"Выводы. Взаимозависимости амплитуд пульсаций, ..."
1643,. Методы. . . Теоретические модели. Количестве...,Методы. Улучшение производительности современн...
1644,. Выводы. В ходе миссии «Кампания К» мы обнару...,Выводы. Мы нашли планету в . AU от одного тран...


## Navec

In [61]:
!pip install wget

In [62]:
import wget

In [63]:
filename = wget.download('https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar')

100% [........................................................................] 26634240 / 26634240

In [64]:
filename

'navec_news_v1_1B_250K_300d_100q (7).tar'

In [65]:
path = 'navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [66]:
def vectorize_sum(words, embeddings):

    embedding_dim = embeddings['example'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')

    num_known_words = 0
    for word in words:
        if word in embeddings:
            num_known_words += 1
            features += embeddings[f'{word}']

    if num_known_words:
        return features / num_known_words
    else: return features

def my_sim(text1, text2, embeddings=navec, kind='cos'):

    v1, v2 = vectorize_sum(text1, embeddings), vectorize_sum(text2, embeddings)
    if np.sum(v1) == 0.0 or np.sum(v2) == 0.0:
        return 0

    return np.abs(np.dot(v1, v2) / norm(v1) / norm(v2))

In [67]:
calc_text_rank_score(test_records, calc_similarity=my_sim, morph=morph)

0it [00:00, ?it/s]

Mean number of sents in summary: 1.9012345679012346
Count: 1620
Ref: методы. мы оценили два современных метода, все из которых основаны на использовании глубоких сверточных нейронных сетей и их способности прогнозировать количество митозов на слайдах цифровой гистопатологии. мы оценили их на новом наборе данных, состоящем из полностью аннотированных цельных изображений тучных клеток кожи собак (cmct) и одного общедоступного набора данных о карциноме молочной железы человека (hmc). сначала мы сравнили количество митозов (mc), предсказанное двумя моделями, с фактическим mc в обоих наборах данных. во-вторых, для набора данных cmct мы сравнили предсказанное с помощью вычислений положение и mc области наивысшей митотической активности с эквивалентными по размеру областями, выбранными восемью ветеринарными патологами.
Hyp: в этой работе нашей целью было сравнить обнаружение интересующей области для алгоритмических подходов с текущим золотым стандартом, то есть ручным выбором обученным и опыт

### Summa

In [68]:
def calc_summa_score(records, summary_part=0.1, lower=True, nrows=1620):
    references = []
    predictions = []

    for i, record in tqdm(records.iterrows()):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        predicted_summary = summarize(text, ratio=summary_part, language='russian').replace("\n", " ")
        if len(predicted_summary)==0:
            predicted_summary='None'
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_summa_score(test_records)

0it [00:00, ?it/s]

Count: 1620
Ref: методы. мы оценили два современных метода, все из которых основаны на использовании глубоких сверточных нейронных сетей и их способности прогнозировать количество митозов на слайдах цифровой гистопатологии. мы оценили их на новом наборе данных, состоящем из полностью аннотированных цельных изображений тучных клеток кожи собак (cmct) и одного общедоступного набора данных о карциноме молочной железы человека (hmc). сначала мы сравнили количество митозов (mc), предсказанное двумя моделями, с фактическим mc в обоих наборах данных. во-вторых, для набора данных cmct мы сравнили предсказанное с помощью вычислений положение и mc области наивысшей митотической активности с эквивалентными по размеру областями, выбранными восемью ветеринарными патологами.
Hyp: в этой работе нашей целью было сравнить обнаружение интересующей области для алгоритмических подходов с текущим золотым стандартом, то есть ручным выбором обученным и опытным патологоанатомом. чтобы установить базовый урове

## Oracle summary

In [69]:
def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()

    score = -1.0
    summaries = []
    for _ in range(n_sentences):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [70]:
def calc_oracle_score(records, nrows=1620, lower=True):
    references = []
    predictions = []
    rouge = Rouge()

    for i, record in tqdm(records.iterrows()):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        if len(predicted_summary)==0:
            predicted_summary='None'
        predictions.append(predicted_summary)

    calc_scores(references, predictions)


calc_oracle_score(test_records)

0it [00:00, ?it/s]

Count: 1620
Ref: методы. мы оценили два современных метода, все из которых основаны на использовании глубоких сверточных нейронных сетей и их способности прогнозировать количество митозов на слайдах цифровой гистопатологии. мы оценили их на новом наборе данных, состоящем из полностью аннотированных цельных изображений тучных клеток кожи собак (cmct) и одного общедоступного набора данных о карциноме молочной железы человека (hmc). сначала мы сравнили количество митозов (mc), предсказанное двумя моделями, с фактическим mc в обоих наборах данных. во-вторых, для набора данных cmct мы сравнили предсказанное с помощью вычислений положение и mc области наивысшей митотической активности с эквивалентными по размеру областями, выбранными восемью ветеринарными патологами.
Hyp: методы. мы сравнили два современных метода, оба из которых направлены на прогнозирование количества митозов в определенной области гистологического препарата. для обоих методов исходный wsi был разделен на множество отдельн

## Extractive RNN

### BPE токенизация

In [71]:
# # python3.9 -m venv venv
# # source venv/bin/activate
# !pip install -U setuptools # setuptools-67.7.1
# !pip install pip==23.0.1 
# !pip install Cython
!pip install youtokentome

In [72]:
import youtokentome as yttm

In [73]:
def train_bpe(records, model_path, model_type="bpe", vocab_size=10000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "w", encoding="utf-8") as temp:
        for i, record in records.iterrows():
            
            text, summary = record['text'], record['summary']
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write(text + "\n")
            temp.write(summary + "\n")
    yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

train_bpe(train_records, "BPE_model.bin")

In [74]:
bpe_processor = yttm.BPE('BPE_model.bin')

In [75]:
vocabulary = bpe_processor.vocab()

In [76]:
def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1620):
    rouge = Rouge()
    for i, record in tqdm(records.iterrows()):
        if i >= nrows:
            break
        text = record["text"]
        summary = record["summary"]

        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        
        record["sentences"] = sentences
        record["oracle_sentences"] = list(sentences_indicies)
        record["oracle_summary"] = oracle_summary

    return records[:nrows]

ext_train_records = add_oracle_summary_to_records(train_records, nrows=3810)
# ext_val_records = add_oracle_summary_to_records(val_records, nrows=256)
ext_test_records = add_oracle_summary_to_records(test_records, nrows=1620)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [77]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [78]:
class BatchIterator():
    def __init__(self, records, vocabulary, batch_size, bpe_processor, shuffle=True, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = len(records)
        self.batch_size = batch_size
        self.bpe_processor = bpe_processor
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(self.num_samples / batch_size))
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device

    def __len__(self):
        return self.batches_count

    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
        
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]

            batch_inputs = []
            batch_outputs = []
            max_sentence_length = 0
            max_sentences = 0
            batch_records = []

            for data_ind in batch_indices:
            
                record = self.records.iloc[data_ind]
             
                batch_records.append(record)
                text = record["text"]
                summary = record["summary"]
                summary = summary.lower() if self.lower else summary
                
                if "sentences" not in record:
                
                    sentences = [sentence.text.lower() if self.lower else sentence.text for sentence in razdel.sentenize(text)][:self.max_sentences]
                    record["sentences"] = sentences
                else:
                    sentences = record["sentences"]
                max_sentences = max(len(sentences), max_sentences)

                # номера предложений, которые в нашем саммари
                if "oracle_sentences" not in record:
                    calc_score = lambda x, y: calc_single_score(x, y, self.rouge)
                    sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=calc_score, lower=self.lower, max_sentences=self.max_sentences)[1]
                else:
                    sentences_indicies = record["oracle_sentences"]

                # inputs - индексы слов в предложении
                inputs = [bpe_processor.encode(sentence)[:self.max_sentence_length] for sentence in sentences]
                max_sentence_length = max(max_sentence_length, max([len(tokens) for tokens in inputs]))

                # получение метки класса предложения
                outputs = [int(i in sentences_indicies) for i in range(len(sentences))]
                batch_inputs.append(inputs)
                batch_outputs.append(outputs)
                

            tensor_inputs = torch.zeros((self.batch_size, max_sentences, max_sentence_length), dtype=torch.long, device=self.device)
            tensor_outputs = torch.zeros((self.batch_size, max_sentences), dtype=torch.float32, device=self.device)


            for i, inputs in enumerate(batch_inputs):
                for j, sentence_tokens in enumerate(inputs):
                    tensor_inputs[i][j][:len(sentence_tokens)] = torch.LongTensor(sentence_tokens)

            for i, outputs in enumerate(batch_outputs):
                tensor_outputs[i][:len(outputs)] = torch.LongTensor(outputs)

            yield {
                'inputs': tensor_inputs,
                'outputs': tensor_outputs,
                'records': batch_records
            }

In [79]:
ext_train_records

,text,summary
0,"VII. ВЫВОДЫ. Наши результаты ясно показывают, ...",Выводы. Изоскалярный бозон не улучшает энергет...
1,. Полученные результаты. . .. Новые данные. Мы...,Полученные результаты. Мы обнаруживаем источни...
2,. Резюме и выводы. На основе глобальных астеро...,"Выводы. И масса, и металличность способствуют ..."
3,. Выводы. Наш верхний предел содержания серы н...,Выводы. Этот верхний предел не дает убедительн...
4,. Полученные результаты. . .. Холодный и теплы...,"Полученные результаты. Одиночные, близкие двой..."
...,...,...
3805,. ВВЕДЕНИЕ. Миссия M Европейского космического...,СЛУЧАЙ. Команда представляет здесь независимое...
3806,. Результаты и обсуждение. . .. Оттоки. Благод...,Полученные результаты. Предыдущее предваритель...
3807,| МЕТОДЫ. . . | Сбор данных: встроенный T∗-диф...,"Методы. Мы представляем новое приобретение, со..."
3808,. Методы интеграции. Для неявного или полунеяв...,Методы. Для исследования эффективности каждого...


In [80]:
train_iterator = BatchIterator(ext_train_records, vocabulary, 4, bpe_processor, device=device)

In [81]:
len(train_iterator)

953

In [82]:
for batch in train_iterator:
    print(batch['records'])
    break

[text         . Полученные результаты. . .. Излучение сантим...
summary      Полученные результаты. Сравнение этих новых да...
sentences    [. полученные результаты. . .., излучение сант...
Name: 3491, dtype: object, text         III. ПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ. Начнем с гл. III A...
summary      Полученные результаты. Мы нашли окончательные ...
sentences    [iii. полученные результаты., начнем с гл. iii...
Name: 1332, dtype: object, text         . Резюме и основные выводы. Необычайный молеку...
summary      Выводы. Состав молекулярного газа остатка весь...
sentences    [. резюме и основные выводы., необычайный моле...
Name: 1956, dtype: object, text         . Обсуждение и выводы. Мы исследовали фотосфер...
summary      Выводы. Наличие полутени не является необходим...
sentences    [. обсуждение и выводы., мы исследовали фотосф...
Name: 3545, dtype: object]


In [83]:
batch['records'][0].keys()

Index(['text', 'summary', 'sentences'], dtype='object')

In [84]:
train_iterator = BatchIterator(ext_train_records, vocabulary, 32, bpe_processor, device=device)
# val_iterator = BatchIterator(ext_val_records, vocabulary, 32, bpe_processor, device=device)
test_iterator = BatchIterator(ext_test_records, vocabulary, 32, bpe_processor, device=device)

### SummaRuNNer


In [85]:
def train_model(model, train_iterator, val_iterator, vocabulary, bpe_processor,
                epochs_count=2, loss_every_nsteps=16, lr=0.001, device_name="cuda"):
    
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))

    device = torch.device(device_name)
    model = model.to(device)

    total_loss = 0
    start_time = time.time()

    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.BCEWithLogitsLoss().to(device)

    for epoch in tqdm(range(epochs_count)):
        for step, batch in tqdm(enumerate(train_iterator)):
            try:
                # print(len(batch['outputs']))
                # if len(batch['outputs'])<32:
                #     break
                model.train()
                logits = model(batch["inputs"]) # Прямой проход
                loss = loss_function(logits, batch["outputs"]) # Подсчёт ошибки
    
                optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
                loss.backward() # Подсчёт градиентов dL/dw
                optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
    
                total_loss += loss.item()
                if step % loss_every_nsteps == 0 and step != 0:
                    val_total_loss = 0
                    val_batch_count = 0
    
                    model.eval()
                    for _, val_batch in enumerate(val_iterator):
    
                        logits = model(val_batch["inputs"]) # Прямой проход
                        val_total_loss += loss_function(logits, batch["outputs"]) # Подсчёт ошибки
                        val_batch_count += 1
    
                    avg_val_loss = val_total_loss/val_batch_count
                    print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
                    total_loss = 0
                    start_time = time.time()
            except:
                    continue
        total_loss = 0
        start_time = time.time()

In [86]:
class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super().__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        # [batch_size, hidden_size]
        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):

        super().__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers,
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)

        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)

        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):

        # [batch_size, seq num, seq_len]
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
        # [batch_size * seq num, seq_len]

        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        # [batch_size *  seq num, seq_len, hidden_size] -> [batch_size, seq num, hidden_size]

        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        # [batch_size, seq num, hidden_size]

        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        # [batch_size, hidden_size]

        # W * h^T
        content = self.content_linear_layer(outputs).squeeze(2) # 1-representation
        # [batch_size, seq num]

        # h^T * W * d
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2) # 2-representation

        # [batch_size, seq num, hidden_size] * [batch_size, hidden_size, 1] = [batch_size, seq num, ]
        return content + salience

In [87]:
len(test_iterator)

51

In [88]:
model = SentenceTaggerRNN(len(vocabulary))
train_model(model, train_iterator, test_iterator, vocabulary, bpe_processor, device_name="cuda")

Trainable params: 3877633


C:\Users\compf\anaconda3\envs\myjenv\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch = 0, Avg Train Loss = 0.3300, Avg val loss = 0.2709, Time = 354.43s
Epoch = 0, Avg Train Loss = 0.2597, Avg val loss = 0.2304, Time = 376.98s
Epoch = 0, Avg Train Loss = 0.4983, Avg val loss = 0.3019, Time = 500.46s
Epoch = 0, Avg Train Loss = 0.2474, Avg val loss = 0.3336, Time = 363.21s


0it [00:00, ?it/s]

Epoch = 1, Avg Train Loss = 0.2396, Avg val loss = 0.2723, Time = 350.78s
Epoch = 1, Avg Train Loss = 0.6438, Avg val loss = 0.2704, Time = 784.06s
Epoch = 1, Avg Train Loss = 0.2078, Avg val loss = 0.2774, Time = 343.92s
Epoch = 1, Avg Train Loss = 0.2134, Avg val loss = 0.3056, Time = 336.86s


In [89]:
def punct_detokenize(text):
    text = text.strip()
    punctuation = ",.!?:;%"
    closing_punctuation = ")]}"
    opening_punctuation = "([}"
    for ch in punctuation + closing_punctuation:
        text = text.replace(" " + ch, ch)
    for ch in opening_punctuation:
        text = text.replace(ch + " ", ch)
    res = [r'"\s[^"]+\s"', r"'\s[^']+\s'"]
    for r in res:
        for f in re.findall(r, text, re.U):
            text = text.replace(f, f[0] + f[2:-2] + f[-1])
    text = text.replace("' s", "'s").replace(" 's", "'s")
    text = text.strip()
    return text


def postprocess(ref, hyp, is_multiple_ref=False, detokenize_after=False, tokenize_after=True):
    if is_multiple_ref:
        reference_sents = ref.split(" s_s ")
        decoded_sents = hyp.split("s_s")
        hyp = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in decoded_sents]
        ref = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in reference_sents]
        hyp = " ".join(hyp)
        ref = " ".join(ref)
    ref = ref.strip()
    hyp = hyp.strip()
    if detokenize_after:
        hyp = punct_detokenize(hyp)
        ref = punct_detokenize(ref)
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    return ref, hyp

def inference_summarunner(model, iterator, top_k=3):
    references = []
    predictions = []

    model.eval()
    for batch in tqdm(test_iterator):

        
        logits = model(batch['inputs'])
        sum_in = torch.argsort(logits, dim=1)[:, -top_k:]
        
        for i in range(len(batch['outputs'])):
            try:
                summary = batch['records'][i]['summary'].lower()
                pred_summary = ' '.join([batch['records'][i]['sentences'][ind] for ind in sum_in.sort(dim=1)[0][i]])
    
                summary, pred_summary= postprocess(summary, pred_summary)
    
                references.append(summary)
                predictions.append(pred_summary)
            except:
                continue

    calc_scores(references, predictions)

inference_summarunner(model, test_iterator, 3)

  0%|          | 0/51 [00:00<?, ?it/s]

Count: 1599
Ref: выводы . используя звезды поля и классические галактические цефеиды , мы обнаружили выше m _ \ odot ммп , более крутое , чем каноническое звездное ммп ассоциаций и молодых скоплений . этот результат согласуется с предсказаниями интегрированного галактического мфм .
Hyp: три рассмотренных здесь статистических метода показывают , что постоянная sfh не согласуется с данными наблюдений , что подтверждает , что история звездообразования уменьшается во времени в тонком диске галактики . впервые мы используем bgm для характеристики молодой популяции классических цефеид и используем самые обновленные границы полосы нестабильности . фоновая музыка позволяет нам правильно разместить модели звездной эволюции в контексте моделирования эволюции млечного пути .
BLEU:  0.3852694103349271
ROUGE:  {'rouge-1': {'f': 0.21219294996009688, 'p': 0.2226801321864533, 'r': 0.23201678770795917}, 'rouge-2': {'f': 0.06429254841583414, 'p': 0.06654392381763716, 'r': 0.07308375177251576}, 'rouge-l'

## MBartForConditionalGeneration

In [90]:
!pip install sentencepiece
!pip install transformers

In [91]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)

model = MBartForConditionalGeneration.from_pretrained(model_name)

In [92]:
model.to('cuda')

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [93]:
for i, batch in tqdm(test_records.iterrows()):
        print(batch)
        print(batch['summary'])
        break

0it [00:00, ?it/s]

text       . Полученные результаты. . .. Суммарная числен...
summary    Полученные результаты. Лучший подход для SP — ...
Name: 0, dtype: object
Полученные результаты. Лучший подход для SP — это перебор всех возможных субпопуляций. Однако это возможно только для небольших групп населения. Для более реалистичных настроек имитация отжига явно превосходит градиентные алгоритмы лишь с ограниченным увеличением вычислительной нагрузки. Наш новый подход LL позволяет обрабатывать очень сложные сценарии кодирования, несмотря на простоту вычислений. Сравнение с существующими


In [98]:
test_records[:500]

,text,summary
0,. Полученные результаты. . .. Суммарная числен...,Полученные результаты. Лучший подход для SP — ...
1,ОБСУЖДЕНИЕ И ВЫВОДЫ. Измерение образования мюо...,Выводы. В пределах экспериментальных и теорети...
2,. Выводы. Эта работа подтверждает наблюдение S...,Выводы. Мы представили функциональное поведени...
3,Полученные результаты. . . Этапы МОЭ. Популяци...,"Полученные результаты. Методы ускорения, реали..."
4,. Полученные результаты. . .. Группа . Мы иска...,Полученные результаты. Мы проводим опрос канди...
...,...,...
495,. Выводы. Определение масс звезд — одна из сло...,"Выводы. Такие события, как Gaiaaye, указывают ..."
496,. Выводы. Мы представляем спектральные наблюде...,"Выводы. Сделан вывод, что смена спектрального ..."
497,| ПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ. На рисунке показаны в...,Полученные результаты. Спектры T-ADC демонстри...
498,. Полученные результаты. Чтобы обнаружить осно...,Полученные результаты. Никаких существенных из...


In [100]:
def inference_summarunner(model, iterator):
    references = []
    predictions = []

    model.eval()
    for i, batch in tqdm(iterator.iterrows()):
        # print(batch)
        summary = batch['summary']
        # print(batch['text'])
        input_ids = tokenizer(
            [batch['text']],
            max_length=1000,
            truncation=True,
            return_tensors="pt",
        )["input_ids"].to("cuda")

        output_ids = model.generate(
            max_length = 500,
            min_length = 40,
            length_penalty = 5,
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )[0]
        pred_summary = tokenizer.decode(output_ids, skip_special_tokens=True)

        references.append(summary)
        predictions.append(pred_summary)

    calc_scores(references, predictions)

inference_summarunner(model, test_records[:100])

0it [00:00, ?it/s]

Count: 100
Ref: Методы. Проблема соблюдения фазовых ограничений при реконструкции изучалась в рамках регуляризованной обратной задачи. Был предложен общий алгоритм регуляризованной реконструкции, позволяющий осуществлять различную совместную реконструкцию частичной визуализации Фурье, визуализации воды и жира и визуализации потока, а также параллельной визуализации (PI) и сжатого зондирования (CS). Поскольку фазовая регуляризованная реконструкция по своей сути невыпуклая и чувствительна к фазовым переворотам в исходном решении, был предложен метод реконструкции, называемый фазовым циклированием, чтобы сделать весь алгоритм инвариантным к фазовым переворотам. Предложенный метод был применен к ретроспективно недостаточно выборочным наборам данных in vivo и сравнен с современными методами реконструкции.
Hyp: Российские ученые проанализировали регуляризацию изображений магнитуды человеческого мозга, полученных с помощью МР-сканера GE Discovery, и выяснили, как это работает в реальном мире.

In [2]:
!pip install gigachat

In [22]:
doc = test_records['text'][0]

In [23]:
doc

'. Полученные результаты. . .. Суммарная численность населения (СП). (i) Грубая сила. Это предпочтительный алгоритм, поскольку он гарантирует достижение наилучших характеристик дискриминации. Однако количество субпопуляций, которые необходимо оценить, увеличивается экспоненциально с количеством нейронов, см. уравнение. , что делает этот алгоритм применимым только для сравнительно небольшого числа нейронов. В этом исследовании, когда это было возможно, мы использовали его в качестве эталона для проверки правильности решений, найденных другими алгоритмами, и для оценки их снижения вычислительных затрат. Возможность получить основную информацию таким способом была наиболее важна для примеров с шумными всплесками, где фактический результат не был известен заранее. (ii) Градиентные алгоритмы. Мы иллюстрируем целесообразность использования градиентных алгоритмов, т.е. доказательство принципа, на примере без шума. На рис. мы использовали тот же пример, что и на рис., снова с S = разные стимул

In [24]:
from gigachat import GigaChat

# Используйте токен, полученный в личном кабинете из поля Авторизационные данные
with GigaChat(credentials='OGMzMDY1MjQtODNiMi00ZDM2LWI5OGItNjJhZmQ3MGNlOWE2OjVmOGQ3ZWU2LTA4NTgtNDdkMC1hMDA5LTE0ODUwODhjMzhlOA==', verify_ssl_certs=False) as giga:
    response = giga.chat("сократи текст:" + doc)
    print(response.choices[0].message.content)

Полученные результаты показали, что грубый force алгоритм предпочтителен, но применим только для небольшого количества нейронов. Для больших объемов данных используются градиентные алгоритмы. В одном из примеров без шума наилучшие характеристики распознавания были получены лучшим отдельным нейроном, а не субпопуляцией.


In [29]:
def calc_gigachat_score(records, nrows=100, lower=True):
    references = []
    predictions = []
    rouge = Rouge()

    for i, record in tqdm(records.iterrows()):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)
        text = record["text"]
        
        with GigaChat(credentials='OGMzMDY1MjQtODNiMi00ZDM2LWI5OGItNjJhZmQ3MGNlOWE2OjVmOGQ3ZWU2LTA4NTgtNDdkMC1hMDA5LTE0ODUwODhjMzhlOA==', verify_ssl_certs=False) as giga:
            response = giga.chat("сократи текст:" + text)
            predicted_summary = response.choices[0].message.content
            
        if len(predicted_summary)==0:
            predicted_summary='None'
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_gigachat_score(test_records)

0it [00:00, ?it/s]

Count: 100
Ref: методы. проблема соблюдения фазовых ограничений при реконструкции изучалась в рамках регуляризованной обратной задачи. был предложен общий алгоритм регуляризованной реконструкции, позволяющий осуществлять различную совместную реконструкцию частичной визуализации фурье, визуализации воды и жира и визуализации потока, а также параллельной визуализации (pi) и сжатого зондирования (cs). поскольку фазовая регуляризованная реконструкция по своей сути невыпуклая и чувствительна к фазовым переворотам в исходном решении, был предложен метод реконструкции, называемый фазовым циклированием, чтобы сделать весь алгоритм инвариантным к фазовым переворотам. предложенный метод был применен к ретроспективно недостаточно выборочным наборам данных in vivo и сравнен с современными методами реконструкции.
Hyp: Было проведено тестирование предложенного метода на трёх приложениях: частичной визуализации Фурье, визуализации воды и жира, и визуализации потоков. В каждом из этих приложений испол